In [1]:
import numpy as np
import pandas as pd
from pycirclize import Circos

In [2]:
df = pd.read_excel('../data/br2f_topographic_data.xlsx', header=0, index_col=0)

In [3]:
# define left hemisphere
brain_regions = df.index.to_numpy()

left_hemisphere = brain_regions[:392]
df_left_hemisphere = df[left_hemisphere]
df_left_hemisphere = df_left_hemisphere.head(392)

In [39]:
df_left_hemisphere.columns[df_left_hemisphere.columns.str.contains('BLA')]

Index(['BLAa', 'BLAp'], dtype='object')

## Hippocampal - Amygdala connectivity plots

Next we will look at three aggregated regions (CEA, MEA, BST) and two non-aggregated regions (AAA, IA) and their connectivity with the hippocampus. For the aggregated regions, we start by summing across the subregions.

In [63]:
# construct regions
CEA_regions = np.array(['CEAm','CEAl','CEAc'])
MEA_regions = np.array(['MEAav','MEAad','MEApv','MEApd'])
BST_regions = np.array([
    'BSTam','BSTfu','BSTv','BSTmg','BSTdm','BSTal','BSTov','BSTju','BSTrh','BSTpr','BSTif','BSTtr','BSTd','BSTse'
])
BLA_regions = np.array(['BLAp','BLAa'])

hippocampus_regions = np.array(['DG','CA3','CA2','CA1v','CA1d','PAR', 'SUBv'])

amygdala_non_aggregated_regions = np.array(['AAA','IA'])

amygdala_aggregated_regions = np.concatenate(
    (CEA_regions, MEA_regions, BST_regions, BLA_regions), axis=0
)

amy_hipp_regions = np.concatenate(
    (amygdala_non_aggregated_regions, amygdala_aggregated_regions, hippocampus_regions), 
    axis=0
)

In [64]:
# reduce the left hemisphere dataframe to only include amygdala and hippocampal regions
df_amy_hipp = df_left_hemisphere[amy_hipp_regions]
df_amy_hipp = df_amy_hipp[df_amy_hipp.index.isin(amy_hipp_regions)]

In [65]:
def consolidate_sum(dataframe, new_col_row_name, subregion_array):
    ret_df = dataframe
    row_sum = ret_df[ret_df.index.isin(subregion_array)].sum(axis=0)
    ret_df.loc[new_col_row_name] = row_sum
    
    # delete extra subregions from rows
    ret_df = ret_df.loc[
    ~ret_df.index.isin(subregion_array)
    ]
    
    # columns
    col_sum = ret_df[subregion_array].sum(axis=1)
    ret_df.loc[:,new_col_row_name] = col_sum
    
    # delete extra subregion from columns
    ret_df = ret_df.drop(subregion_array, axis=1)
    return ret_df

def consolidate_average(dataframe, new_col_row_name, subregion_array):
    ret_df = dataframe
    row_sum = ret_df[ret_df.index.isin(subregion_array)].mean(axis=0)
    ret_df.loc[new_col_row_name] = row_sum
    
    # delete extra subregions from rows
    ret_df = ret_df.loc[
    ~ret_df.index.isin(subregion_array)
    ]
    
    # columns
    col_sum = ret_df[subregion_array].mean(axis=1)
    ret_df.loc[:,new_col_row_name] = col_sum
    
    # delete extra subregion from columns
    ret_df = ret_df.drop(subregion_array, axis=1)
    return ret_df

In [66]:
df_amy_hipp = consolidate_sum(df_amy_hipp, "CEA", CEA_regions)
df_amy_hipp = consolidate_sum(df_amy_hipp, "MEA", MEA_regions)
df_amy_hipp = consolidate_sum(df_amy_hipp, "BST", BST_regions)
df_amy_hipp = consolidate_sum(df_amy_hipp, "BLA", BLA_regions)

/var/folders/4k/31jdbs595pn4tslnsm4nxmzm0000gq/T/ipykernel_56065/3939621999.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ret_df.loc[:,new_col_row_name] = col_sum
/var/folders/4k/31jdbs595pn4tslnsm4nxmzm0000gq/T/ipykernel_56065/3939621999.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ret_df.loc[:,new_col_row_name] = col_sum
/var/folders/4k/31jdbs595pn4tslnsm4nxmzm0000gq/T/ipykernel_56065/3939621999.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

In [67]:
df_amy_hipp

,AAA,IA,DG,CA3,CA2,CA1v,CA1d,PAR,SUBv,CEA,MEA,BST,BLA
Abbr.,,,,,,,,,,,,,
AAA,0,0,0,0,0,4,0,0,0,6,0,31,0
IA,0,0,0,0,0,0,0,0,0,4,0,0,4
DG,0,0,0,6,0,0,0,0,0,0,0,0,0
CA3,0,0,3,0,6,6,6,0,3,0,0,0,0
CA2,0,0,4,4,0,4,0,2,3,0,0,0,0
CA1v,3,1,2,1,2,0,0,4,7,2,4,16,5
CA1d,0,0,2,0,0,0,0,1,0,0,0,0,0
SUBv,1,5,3,3,3,3,0,5,0,12,6,28,4
PAR,0,0,2,0,0,1,0,0,1,0,0,0,0


In [68]:
# remove self-connections (set diagonal to 0)
for i in range(len(df_amy_hipp.index)):
    df_amy_hipp.iloc[i,i] = 0

In [69]:
# we need to remove connections within the hypothalamus--
# we're only interested in hippocampal-hypothalamus connectivity

amygdala_regions = np.array(['AAA','IA', 'CEA','MEA','BST','BLA'])
for subregion_i in range(len(amygdala_regions)):
    for subregion_j in range(len(amygdala_regions)):
        df_amy_hipp.loc[
        amygdala_regions[subregion_i], 
        amygdala_regions[subregion_j]
        ] = 0

In [70]:
df_amy_hipp

,AAA,IA,DG,CA3,CA2,CA1v,CA1d,PAR,SUBv,CEA,MEA,BST,BLA
Abbr.,,,,,,,,,,,,,
AAA,0,0,0,0,0,4,0,0,0,0,0,0,0
IA,0,0,0,0,0,0,0,0,0,0,0,0,0
DG,0,0,0,6,0,0,0,0,0,0,0,0,0
CA3,0,0,3,0,6,6,6,0,3,0,0,0,0
CA2,0,0,4,4,0,4,0,2,3,0,0,0,0
CA1v,3,1,2,1,2,0,0,4,7,2,4,16,5
CA1d,0,0,2,0,0,0,0,1,0,0,0,0,0
SUBv,1,5,3,3,3,3,0,0,0,12,6,28,4
PAR,0,0,2,0,0,1,0,0,0,0,0,0,0


In [71]:
#clear out intrinsic connections between hippocampus_regions
for subregion_i in range(len(hippocampus_regions)):
    for subregion_j in range(len(hippocampus_regions)):
        df_amy_hipp.loc[
        hippocampus_regions[subregion_i], 
        hippocampus_regions[subregion_j]
        ] = 0

In [72]:
df_amy_hipp

,AAA,IA,DG,CA3,CA2,CA1v,CA1d,PAR,SUBv,CEA,MEA,BST,BLA
Abbr.,,,,,,,,,,,,,
AAA,0,0,0,0,0,4,0,0,0,0,0,0,0
IA,0,0,0,0,0,0,0,0,0,0,0,0,0
DG,0,0,0,0,0,0,0,0,0,0,0,0,0
CA3,0,0,0,0,0,0,0,0,0,0,0,0,0
CA2,0,0,0,0,0,0,0,0,0,0,0,0,0
CA1v,3,1,0,0,0,0,0,0,0,2,4,16,5
CA1d,0,0,0,0,0,0,0,0,0,0,0,0,0
SUBv,1,5,0,0,0,0,0,0,0,12,6,28,4
PAR,0,0,0,0,0,0,0,0,0,0,0,0,0


In [73]:
# remove rows of all zero
df_boop = df_amy_hipp.loc[:,(df_amy_hipp!=0).any()]

In [74]:
(df_boop.T!=0).any()

Abbr.
AAA      True
IA      False
DG      False
CA3     False
CA2     False
CA1v     True
CA1d    False
SUBv     True
PAR     False
CEA     False
MEA      True
BST      True
BLA      True
dtype: bool

In [75]:
df_boop = df_boop.loc[(df_boop.T!=0).any(),:]

In [76]:
df_boop

,AAA,IA,CA3,CA2,CA1v,CA1d,PAR,SUBv,CEA,MEA,BST,BLA
Abbr.,,,,,,,,,,,,
AAA,0,0,0,0,4,0,0,0,0,0,0,0
CA1v,3,1,0,0,0,0,0,0,2,4,16,5
SUBv,1,5,0,0,0,0,0,0,12,6,28,4
MEA,0,0,0,0,5,0,3,6,0,0,0,0
BST,0,0,0,0,8,0,0,3,0,0,0,0
BLA,0,0,7,4,8,2,9,7,0,0,0,0


In [77]:
# re-order rows for more coherent circular visualization
df_amy_hipp_clean = df_boop.loc[['CA1v','SUBv','AAA','MEA','BST','BLA']]

In [78]:
circos_amy_hipp_extrinsic_only = Circos.initialize_from_matrix(df_amy_hipp_clean, 
                                                               space=5,
                                                               cmap="jet")
# cmap options are jet, viridis, tab10, etc
circos_amy_hipp_extrinsic_only.savefig(
    "../output/hippocampus_amygdala_extrinsic_summed_connectivity.svg")
